# Loading a Using a Model

In this notebook we demonstrate how we can load a saved model without other dependencies and without re-defining the whole Graph network. We will apply the feature encoding on sample text and score it using the loaded model.

In [1]:
import numpy as np
import tensorflow as tf
import json
from processing import text as text_prepro

## Define feature encoders

Load vocabularies and re-construct pre-processing pipelines and feature extraction used on the training dataset.

In [2]:
# load vocabularies
with open('datasets/semantic_vocab.json') as f:
    word_indexes = json.load(f)
with open('datasets/letter_vocab.json') as f:
    char_indexes = json.load(f)

In [3]:
# pre-processing pipeline
pipeline = [
    # convert all letters to lowercase
    text_prepro.to_lower,
    # transliterate non-english letters
    text_prepro.transliterate,
    # strip tags (@ and #) from words
    text_prepro.remove_tags,
    # tokenize URLs into "__URL__"
    text_prepro.tokenize_url,
    # Keep alphanumeric characters only
    text_prepro.alphanum
]

In [4]:
# define the prepro as a functiom
def semantic_prepro(text):
    for pipe in pipeline:
        text = pipe(text)
    return text

In [5]:
# semantic vector mapper
def semantic_vector(text):
    text = semantic_prepro(text)
    vector = np.zeros((len(word_indexes),), dtype=np.float32)
    for w in text.split():
        ind = word_indexes.get(w)
        if ind is not None:
            vector[ind] = 1.
    return vector

In [6]:
# character mapper
def letter_vector(text):
    vector = np.zeros((len(char_indexes),), dtype=np.float32)
    for i in xrange(0, len(text), 2):
        c = text[i:i + 2]
        i = char_indexes.get(c)
        if i is not None:
            vector[i] = 1.
    return vector

In [7]:
def encode_text(text):
    s = semantic_vector(text)
    l = letter_vector(text)
    return np.concatenate([s, l])

## Load the saved model

We first load the saved Graph use `tf.train.import_meta_graph`, this loads the full NN we previously built for training. We then load the model checkpoint using the `saver` object retrieved from the saved graph. This will load and initialize all the weights and biases we saved in our training session.

In [8]:
# restore meta graph
saver = tf.train.import_meta_graph('models/mlp-1600.meta')

In [9]:
# initialize a new session
session = tf.Session()

In [10]:
# restore latest checkpoint
saver.restore(session, 'models/mlp-1600')

INFO:tensorflow:Restoring parameters from models/mlp-1600


In [11]:
# get default graph (to which the meta graph has been loaded by default)
g = tf.get_default_graph()

In [12]:
# check the loaded collection keys
g.get_all_collection_keys()

[u'inputs',
 u'trainable_variables',
 u'variables',
 u'predictor',
 u'train_op',
 u'summaries']

In [13]:
# inputs placeholder
x = g.get_collection('inputs')[0]

In [14]:
# predictor
predictor = g.get_collection('predictor')[0]

## Scoring records

After loading the model, we can now feed it values to get a score. We encode the given text into a feature vector and feed it to the network to get a prediction.

In [15]:
# score sample text
session.run(predictor, feed_dict={x: [encode_text(u'hello world!')]})

array([[ 0.56316912]], dtype=float32)

In [16]:
session.run(predictor, feed_dict={x: [encode_text(u"I'm so happy I trained my first Neural Network :D")]})

array([[ 0.54647446]], dtype=float32)

In [17]:
session.run(predictor, feed_dict={x: [encode_text(u"shut up you idiot...")]})

array([[ 0.61093313]], dtype=float32)